In [233]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from amplpy import AMPL, ampl_notebook
from joblib import load, dump
import sys
sys.path.append("../src")
from utils import cost_prediction

import warnings
warnings.filterwarnings("ignore")
pd.set_option("future.no_silent_downcasting", True) # Prevent silent data type changes during operations for future compatibility

Instantiate the AMPL object

In [234]:
ampl = ampl_notebook(
    modules=["highs","coin"], # solver to be used
    license_uuid="d2b35988-195b-44a4-bca2-fd80a770586f" # license key
)

Licensed to AMPL Community Edition License for <hchoi0309@gmail.com>.


In [235]:
%%ampl_eval
reset;

Define the model

In [236]:
%%ampl_eval
# Sets
set ROUTES;
set AIRPLANE_TYPES;

# Parameters
param a {ROUTES};          # Base fare price coefficient
param b {ROUTES};          # Price elasticity coefficient
param r {ROUTES};          # Yearly effect coefficient
param d {ROUTES};          # Quarter 1 effect coefficient
param e {ROUTES};          # Quarter 2 effect coefficient
param z {ROUTES};          # Quarter 3 effect coefficient
param h {ROUTES};          # Quarter 4 effect coefficient
param c {ROUTES};          # Cost per seat
param N {ROUTES};          # Predicted daily passengers
param C {AIRPLANE_TYPES};  # Capacity of each airplane type
param R {AIRPLANE_TYPES};  # Total airplanes of each type
param t;                   # Target market share (fraction)
param year;                # Current year
param active_quarter;      # Active quarter (1, 2, 3, or 4)

# Variables
var x {ROUTES} >= 0, integer;      # Daily number of seats sold
var r_assign {ROUTES, AIRPLANE_TYPES} >= 0, integer;  # Number of airplanes assigned per route
var q {1..4} binary;               # Quarter indicator variables

# Objective
maximize Profit:
    sum {i in ROUTES} (
        x[i] * (
            a[i] * exp(-b[i] * x[i] + r[i] * year +
                       d[i] * q[1] + e[i] * q[2] +
                       z[i] * q[3] + h[i] * q[4]
            ) - c[i]
        )
    );

# Constraints
s.t. MarketShare {i in ROUTES}:
    x[i] <= t * N[i];

s.t. MinCapacity {i in ROUTES}:
    x[i] >= sum {k in AIRPLANE_TYPES} 0.6 * C[k] * r_assign[i, k];

s.t. MaxCapacity {i in ROUTES}:
    x[i] <= sum {k in AIRPLANE_TYPES} C[k] * r_assign[i, k];

s.t. FleetLimit {k in AIRPLANE_TYPES}:
    sum {i in ROUTES} r_assign[i, k] <= 4 * R[k];

s.t. ActiveQuarter:
    sum {qtr in 1..4} q[qtr] = 1;

s.t. QuarterActive:
    q[active_quarter] = 1;

In [237]:
allegiant_df = pd.read_csv("../data/allegiant_routes_info_transformed.csv")
allegiant_df["year"] = 2019
allegiant_df["quarter"] = 4
allegiant_df.head()

,distance,airport_1,airport_2,daily_passengers,state_1,city_1,state_2,city_2,population_1,density_1,population_2,density_2,lat_1,lon_1,lat_2,lon_2,year,quarter
0,1342,AUS,IAD,222.211864,TX,Austin,DC,Washington,1905945.0,1154.1,5116378.0,4235.7,30.264979,-97.746598,38.892062,-77.019912,2019,4
1,1340,AUS,IAD,222.211864,TX,Austin,DC,Washington,1905945.0,1154.1,5116378.0,4235.7,30.264979,-97.746598,38.892062,-77.019912,2019,4
2,587,BNA,IAD,108.618644,TN,Nashville,DC,Washington,1177657.0,555.4,5116378.0,4235.7,36.166687,-86.779932,38.892062,-77.019912,2019,4
3,588,BNA,IAD,108.618644,TN,Nashville,DC,Washington,1177657.0,555.4,5116378.0,4235.7,36.166687,-86.779932,38.892062,-77.019912,2019,4
4,612,BNA,PIE,42.760000,TN,Nashville,FL,Tampa,1177657.0,555.4,2861173.0,1320.9,36.166687,-86.779932,37.860600,-78.804199,2019,4


In [238]:
preprocessing_pipeline = load("../models/preprocessing_pipeline.joblib")
feature_names = load("../models/feature_names.joblib")
allegiant_df = preprocessing_pipeline.transform(allegiant_df)
allegiant_df = pd.DataFrame(allegiant_df, columns=feature_names)
allegiant_df.head()

,quarter_1,quarter_2,quarter_3,quarter_4,city_1,city_2,airport_1,airport_2,state_1,state_2,year,distance,population_1,density_1,population_2,density_2,lat_1,lon_1,lat_2,lon_2
0,-0.592028,-0.571486,-0.576259,1.755523,-0.042234,-0.013962,-0.032676,-0.502082,-0.099588,-0.003964,1.204085,0.216246,-0.694892,-0.715648,-0.172653,0.043854,-1.706588,-0.522360,0.127185,0.839496
1,-0.592028,-0.571486,-0.576259,1.755523,-0.042234,-0.013962,-0.032676,-0.502082,-0.099588,-0.003964,1.204085,0.213401,-0.694892,-0.715648,-0.172653,0.043854,-1.706588,-0.522360,0.127185,0.839496
2,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,-0.013962,-0.117519,-0.502082,-1.044005,-0.003964,1.204085,-0.857693,-0.826930,-0.916193,-0.172653,0.043854,-0.465810,0.186579,0.127185,0.839496
3,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,-0.013962,-0.117519,-0.502082,-1.044005,-0.003964,1.204085,-0.856271,-0.826930,-0.916193,-0.172653,0.043854,-0.465810,0.186579,0.127185,0.839496
4,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,0.545563,-0.117519,-1.082934,-1.044005,1.068477,1.204085,-0.822132,-0.826930,-0.916193,-0.538597,-0.800943,-0.465810,0.186579,-0.091321,0.745495


In [239]:
demand_model = load("../models/demand_prediction_model.joblib")
allegiant_df["predicted_passengers"] = demand_model.predict(allegiant_df)
display(allegiant_df.head())

,quarter_1,quarter_2,quarter_3,quarter_4,city_1,city_2,airport_1,airport_2,state_1,state_2,year,distance,population_1,density_1,population_2,density_2,lat_1,lon_1,lat_2,lon_2,predicted_passengers
0,-0.592028,-0.571486,-0.576259,1.755523,-0.042234,-0.013962,-0.032676,-0.502082,-0.099588,-0.003964,1.204085,0.216246,-0.694892,-0.715648,-0.172653,0.043854,-1.706588,-0.522360,0.127185,0.839496,314.190399
1,-0.592028,-0.571486,-0.576259,1.755523,-0.042234,-0.013962,-0.032676,-0.502082,-0.099588,-0.003964,1.204085,0.213401,-0.694892,-0.715648,-0.172653,0.043854,-1.706588,-0.522360,0.127185,0.839496,314.190399
2,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,-0.013962,-0.117519,-0.502082,-1.044005,-0.003964,1.204085,-0.857693,-0.826930,-0.916193,-0.172653,0.043854,-0.465810,0.186579,0.127185,0.839496,220.477982
3,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,-0.013962,-0.117519,-0.502082,-1.044005,-0.003964,1.204085,-0.856271,-0.826930,-0.916193,-0.172653,0.043854,-0.465810,0.186579,0.127185,0.839496,220.477982
4,-0.592028,-0.571486,-0.576259,1.755523,-0.162528,0.545563,-0.117519,-1.082934,-1.044005,1.068477,1.204085,-0.822132,-0.826930,-0.916193,-0.538597,-0.800943,-0.465810,0.186579,-0.091321,0.745495,55.914410


In [215]:
allegiant_df["cost"] = cost_prediction(allegiant_df, 2019)
allegiant_df["predicted_passengers"] = 1000
allegiant_df.head()

,a,b,year_coeff,quarter_1_coef,quarter_2_coef,quarter_3_coef,quarter_4_coef,nsmiles,Route,cost,predicted_passengers
2,3.700000e-10,0.096112,0.013684,-9.790509e-03,0.042001,-0.006709,-0.025502,587,BNA_IAD,102.76609,1000
3,3.700000e-10,0.096112,0.013684,-9.790509e-03,0.042001,-0.006709,-0.025502,588,BNA_IAD,102.94116,1000
6,3.070000e-26,0.960332,0.034155,-2.780000e-17,0.067533,0.084137,-0.151670,740,BOS_GRR,129.55180,1000
10,8.400000e-19,0.093016,0.023508,-4.713162e-02,0.089550,-0.047276,0.004858,917,BOS_SAV,160.53919,1000
11,1.530000e-15,0.016181,0.019596,4.956549e-02,0.070588,-0.153771,0.033618,1240,BOS_SRQ,217.08680,1000


In [216]:
passengers_df = allegiant_df[["Route", "nsmiles"]].rename({"nsmiles": "distance"}, axis=1)
passengers_df["airport_1"] = passengers_df["Route"].str.split("_").str[0]
passengers_df["airport_2"] = passengers_df["Route"].str.split("_").str[1]
passengers_df = passengers_df.drop(["Route"], axis=1)

In [217]:
fleet_df = pd.read_csv("../data/allegiant_fleet.csv")
display(fleet_df)

,Aircraft Type,Number of Aircraft,Seats
0,Airbus A319-100,34,156
1,Airbus A320-200 A,14,177
2,Airbus A320-200 B,78,186
3,Boeing 737 MAX 200,2,190


Define the problem

In [218]:
ampl.set["ROUTES"] = allegiant_df["Route"].drop_duplicates()
ampl.param["a"] = allegiant_df.set_index("Route")["a"] / 1e9
ampl.param["b"] = allegiant_df.set_index("Route")["b"]
ampl.param["r"] = allegiant_df.set_index("Route")["year_coeff"]
ampl.param["d"] = allegiant_df.set_index("Route")["quarter_1_coef"]
ampl.param["e"] = allegiant_df.set_index("Route")["quarter_2_coef"]
ampl.param["z"] = allegiant_df.set_index("Route")["quarter_3_coef"]
ampl.param["h"] = allegiant_df.set_index("Route")["quarter_4_coef"]
ampl.param["c"] = allegiant_df.set_index("Route")["cost"]

ampl.param["N"] = allegiant_df.set_index("Route")["predicted_passengers"]

ampl.set["AIRPLANE_TYPES"] = fleet_df["Aircraft Type"]
ampl.param["C"] = fleet_df.groupby("Aircraft Type")["Seats"].first()
ampl.param["R"] = fleet_df.groupby("Aircraft Type")["Number of Aircraft"].first()

ampl.param["t"] = 0.8
ampl.param["year"] = 2019
ampl.param["active_quarter"] = 4

In [219]:
# Add these debugging statements after setting parameters
print("Model Parameters:")
print(f"Market share target (t): {ampl.param['t'].value()}")
print("\nSample Routes Data:")
print("Route | a | b | cost | N")
for route in list(ampl.set['ROUTES'])[:5]:  # First 5 routes
    print(f"{route} | {ampl.param['a'][route]} | {ampl.param['b'][route]} | {ampl.param['c'][route]} | {ampl.param['N'][route]}")

print("\nAirplane Types:")
print("Type | Capacity | Count")
for type_ in ampl.set['AIRPLANE_TYPES']:
    print(f"{type_} | {ampl.param['C'][type_]} | {ampl.param['R'][type_]}")

Model Parameters:
Market share target (t): 0.8

Sample Routes Data:
Route | a | b | cost | N
BNA_IAD | 3.7e-19 | 0.096112047 | 102.94116 | 1000
BOS_GRR | 3.07e-35 | 0.960331876 | 129.55180000000001 | 1000
BOS_SAV | 8.4e-28 | 0.093016345 | 160.53919 | 1000
BOS_SRQ | 1.53e-24 | 0.016181239 | 217.0868 | 1000
FLL_BNA | 1.2900000000000001e-23 | 0.238082647 | 141.10642 | 1000

Airplane Types:
Type | Capacity | Count
Airbus A319-100 | 156 | 34
Airbus A320-200 A | 177 | 14
Airbus A320-200 B | 186 | 78
Boeing 737 MAX 200 | 190 | 2


In [220]:
# # Define data for the problem
# routes = ['AUS_IAD', 'LAX_ORD', 'JFK_LHR', 'SEA_SFO']
# airplane_types = ['type1', 'type2']

# # Parameter values
# a = {'AUS_IAD': 100, 'LAX_ORD': 120, 'JFK_LHR': 150, 'SEA_SFO': 90}
# b = {'AUS_IAD': 0.01, 'LAX_ORD': 0.02, 'JFK_LHR': 0.03, 'SEA_SFO': 0.015}
# r = {'AUS_IAD': 0.1, 'LAX_ORD': 0.15, 'JFK_LHR': 0.2, 'SEA_SFO': 0.12}
# d = {'AUS_IAD': 0.05, 'LAX_ORD': 0.06, 'JFK_LHR': 0.07, 'SEA_SFO': 0.04}
# e = {'AUS_IAD': 0.04, 'LAX_ORD': 0.05, 'JFK_LHR': 0.06, 'SEA_SFO': 0.03}
# z = {'AUS_IAD': 0.03, 'LAX_ORD': 0.04, 'JFK_LHR': 0.05, 'SEA_SFO': 0.025}
# h = {'AUS_IAD': 0.02, 'LAX_ORD': 0.03, 'JFK_LHR': 0.04, 'SEA_SFO': 0.02}
# c = {'AUS_IAD': 50, 'LAX_ORD': 60, 'JFK_LHR': 70, 'SEA_SFO': 45}
# N = {'AUS_IAD': 200, 'LAX_ORD': 250, 'JFK_LHR': 300, 'SEA_SFO': 180}
# C = {'type1': 150, 'type2': 200}
# R = {'type1': 5, 'type2': 3}

# # Load data into AMPL
# ampl.set['ROUTES'] = routes
# ampl.set['AIRPLANE_TYPES'] = airplane_types
# ampl.param['a'] = a
# ampl.param['b'] = b
# ampl.param['r'] = r
# ampl.param['d'] = d
# ampl.param['e'] = e
# ampl.param['z'] = z
# ampl.param['h'] = h
# ampl.param['c'] = c
# ampl.param['N'] = N
# ampl.param['C'] = C
# ampl.param['R'] = R
# ampl.param['t'] = 0.3
# ampl.param['year'] = 2024
# ampl.param['active_quarter'] = 3

In [221]:
%%ampl_eval
option solver highs
solve;

In [222]:
# Retrieve results
x = ampl.getVariable("x").getValues().toPandas()
r_assign = ampl.getVariable("r_assign").getValues().toPandas()
profit = ampl.getObjective("Profit").value()

# Display results
print("Optimal Seat Sales (x):")
print(x)

print("\nAirplane Assignments (r_assign):")
print(r_assign)

print(f"\nTotal Profit: {profit}")

Optimal Seat Sales (x):
         x.val
AUS_BUR      0
AUS_CMH      0
AUS_HOU      0
AUS_IAH      0
AUS_ISP      0
...        ...
SNA_GEG      0
SNA_MFR      0
SNA_MSO      0
SRQ_IAD      0
VPS_BWI      0

[130 rows x 1 columns]

Airplane Assignments (r_assign):
                            r_assign.val
index0  index1                          
AUS_BUR Airbus A319-100                0
        Airbus A320-200 A              0
        Airbus A320-200 B              0
        Boeing 737 MAX 200             0
AUS_CMH Airbus A319-100                0
...                                  ...
SRQ_IAD Boeing 737 MAX 200             0
VPS_BWI Airbus A319-100                0
        Airbus A320-200 A              0
        Airbus A320-200 B              0
        Boeing 737 MAX 200             0

[520 rows x 1 columns]

Total Profit: 0.0


In [223]:
%%ampl_eval
# Add suffix declarations to your model
suffix up;     # Upper sensitivity value
suffix down;   # Lower sensitivity value
suffix current;# Current parameter value

option sensitivity_report 1;  # Enable sensitivity reporting
solve;

# Now you can access sensitivity ranges for parameters
display {i in ROUTES} a[i].current;
display {i in ROUTES} a[i].down;
display {i in ROUTES} a[i].up;

Cannot find "highssolve"
	line 11 offset 311
	Bad suffix .current -- a cannot have a suffix.
	context:  display {i in ROUTES}  >>> a[i].current; <<< 
0 [*] :=
AUS_BUR 0   BOS_CMH 0   BOS_RDU 0   LAS_IAD 0   LAX_MSO 0   LAX_SWF 0
AUS_CMH 0   BOS_DAL 0   BOS_RIC 0   LAS_ISP 0   LAX_MSP 0   LAX_SYR 0
AUS_HOU 0   BOS_DAY 0   BOS_ROC 0   LAS_JFK 0   LAX_MSY 0   LAX_TPA 0
AUS_IAH 0   BOS_DTW 0   BOS_RSW 0   LAS_MIA 0   LAX_OMA 0   LAX_TUL 0
AUS_ISP 0   BOS_EWR 0   BOS_SAT 0   LAS_ONT 0   LAX_ORF 0   LAX_TUS 0
AUS_JFK 0   BOS_GRR 0   BOS_SAV 0   LAS_ORF 0   LAX_PBI 0   MDW_SAV 0
AUS_LAX 0   BOS_HOU 0   BOS_SDF 0   LAS_PHF 0   LAX_PDX 0   MDW_TYS 0
AUS_LGB 0   BOS_IAD 0   BOS_SLC 0   LAS_SJC 0   LAX_PHF 0   MYR_SWF 0
AUS_MIA 0   BOS_ISP 0   BOS_SRQ 0   LAS_SNA 0   LAX_PHL 0   PHX_BWI 0
AUS_ONT 0   BOS_JFK 0   BOS_SWF 0   LAS_TPA 0   LAX_PIT 0   PHX_FSD 0
AUS_PIE 0   BOS_LAX 0   BOS_TUS 0   LAX_BWI 0   LAX_PSC 0   PHX_IAD 0
AUS_SNA 0   BOS_LGB 0   FLL_BNA 0   LAX_FLL 0   LAX_RDU 0   PHX_ICT 0
A